# Parameter exploration

Purpose: Run the simulation with varying parameters and characterize the effects of those parameters

- Option 1: psyrun
- Option 2: doit

(Discuss differences between random search and grid search)